In [1]:
import numpy as np
from numpy.linalg import norm
import os
import time
import pickle
from PIL import Image

In [6]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [7]:
model = VGG16(weights='imagenet', include_top=False, input_shape =(224, 224, 3))

In [31]:
def extract_features(path, model): 
    img = image.load_img(path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [32]:
def store_embeddings(df):
    
    
    data1, data2, labels = [], [], []
    herocat, stylingcat = [],[]

    for _,row in df.iterrows():
        img_one = extract_features('data/'+ row['HeroID'], model)

        img_two = extract_features('data/'+ row['StylingID'], model)

        label = row['Match']
        labels.append(label)
        

        data1.append(img_one)
        data2.append(img_two)

        
    labels = np.array(labels)
    data1 = np.array(data1)
    data2 = np.array(data2)


    return data1, data2, labels

In [33]:
import pandas as pd

In [34]:
train = pd.read_csv('data/pracdat.csv')

In [35]:
traindata1, traindata2, trainlabel = store_embeddings(train)

In [36]:
traindata2[0].shape

(25088,)

In [37]:
traindata1.shape

(60, 25088)

In [38]:
trainlabel.shape

(60,)

In [16]:
model.Trainable = False

In [17]:
from keras.layers import Input, Embedding, Dense, Activation, Dot, Reshape, concatenate, Flatten

In [18]:
from keras.models import Model

In [19]:

#img 1 pass through 2 relu layers
outfit1_img = Input(name = 'outfit1img', shape = (25088,))

    
#img 2 pass through 2 relu layers
outfit2_img = Input(name = 'outfit2img', shape = (25088,))

#two Dense for item1
outfit1imgdense = Dense(256, activation='relu')(outfit1_img)
x1 = Dense(256, activation='relu')(outfit1imgdense)

#two Dense for item2
outfit2imgdense = Dense(256, activation='relu')(outfit2_img)
x2 = Dense(256, activation='relu')(outfit2imgdense)

# merge features using dot layer
    
merged = Dot(name = 'dot_product', normalize = False, axes = 1)([x1,x2]) 
#reshape for input into sigmoid
merged = Reshape(target_shape = [1])(merged)
out = Dense(1, activation = 'sigmoid')(merged)


model1 = Model(inputs =[outfit1_img, outfit2_img], outputs = out)
model1.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#commence training
model1.fit([traindata1, traindata2], trainlabel, epochs=30, batch_size=20)
   



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
60/60 [==============================] - 2s 30ms/step - loss: 0.6931 - acc: 0.5333
Epoch 2/30
60/60 [==============================] - 1s 9ms/step - loss: 0.6884 - acc: 0.5500
Epoch 3/30
60/60 [==============================] - 1s 9ms/step - loss: 0.6545 - acc: 0.5000
Epoch 4/30
60/60 [==============================] - 1s 9ms/step - loss: 0.5411 - acc: 0.5000
Epoch 5/30
60/60 [==============================] - 1s 9ms/step - loss: 0.4304 - acc: 0.5000
Epoch 6/30
60/60 [==============================] - 1s 9ms/step - loss: 0.3661 - acc: 0.7500
Epoch 7/30
60/60 [==============================] - 1s 14ms/step - loss: 0.3467 - acc: 0.9500
Epoch 8/30
60/60 [==============================] - 1s 10ms/step - loss: 0.3440 - acc: 1.0000
Epoch 9/30
60/60 [==============================] - 1s 9ms/step - loss: 0.3428 - acc: 1.0000
Epoch 10/30
60/60 [==============================] - 1s 13ms/s

model()

In [20]:
predict_df = pd.read_csv('data/pracdatatest.csv')

In [21]:
testdata1, testdata2, testlabel = store_embeddings(predict_df)

In [22]:
prediction = model1.predict([testdata1, testdata2])

In [23]:
prediction.shape

(15, 1)

In [24]:
print(prediction[0])

[0.9481745]


In [25]:
print(prediction[14])

[0.47856745]


In [26]:
print(prediction[13])

[0.9666852]


In [27]:
print(prediction[8])

[0.4790791]


In [88]:
model1.save('outfitrecsys1.h5')

In [28]:
pickle.dump(model1, open('data/model1.pickle','wb'))

In [30]:
# Saving the model 

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")